In [2]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
import torch
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter
import numpy as np
from textblob import TextBlob
import pickle

tqdm.pandas()

In [3]:
!ps aux | grep 130318

datht     50812  0.0  0.0  13328  3152 pts/1    Ss+  22:28   0:00 /bin/bash -c ps aux | grep 130318
datht     50814  0.0  0.0  14868  1060 pts/1    S+   22:28   0:00 grep 130318


In [4]:
import dgl
from dgl.data.utils import save_graphs, load_graphs

In [5]:
dataset = 'PolitiFact'

In [6]:
df_train = pd.read_csv(f'/home/datht/GAGA/GET/formatted_data/declare/{dataset}/mapped_data/5fold/train_0.tsv', delimiter='\t')
df_test = pd.read_csv(f'/home/datht/GAGA/GET/formatted_data/declare/{dataset}/mapped_data/5fold/test_0.tsv', delimiter='\t')
df_dev = pd.read_csv(f'/home/datht/GAGA/GET/formatted_data/declare/{dataset}/mapped_data/dev.tsv', delimiter='\t')

In [7]:
df = pd.concat([df_train, df_test, df_dev])

In [8]:
# nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').cuda()
# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
nli_model = AutoModelForSequenceClassification.from_pretrained('textattack/bert-base-uncased-MNLI').cuda()
tokenizer = AutoTokenizer.from_pretrained('textattack/bert-base-uncased-MNLI')

def score(sequence1, sequence2):
    try:
        # run through model pre-trained on MNLI
        x = tokenizer.encode(sequence1.strip(), sequence2.strip(), return_tensors='pt',
                          truncation_strategy='only_first')
        logits = nli_model(x.cuda())[0]

        # we throw away "neutral" (dim 1) and take the probability of
        # "entailment" (2) as the probability of the label being true 
        entail_contradiction_logits = logits[:,[0,2]]
        probs = entail_contradiction_logits.softmax(dim=1)
        prob_label_is_true = probs[:,1]
        return prob_label_is_true.detach().item()
    except:
        print(sequence1, sequence2)
        return 0.5

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
df['score'] = df.progress_apply(lambda row : score(row.claim_text, row.evidence), axis=1)

 22%|████████▉                               | 391/1755 [00:09<00:34, 39.67it/s]

the first cases of covid19 were reported in the united states and south korea on the same day south australia northern territory western australia tasmania ... weekly case numbers change jurisdiction average daily cases 08 nov 2022 change average cases compared previous week national 7809 473 nsw 2469 540 vic 2213 577 qld 758 363 wa 1074 141 sa 819 692 tas 265 641 act 163 424 nt 48 363 source jurisdictional reporting 09 nov 2022 please note due rounding jurisdiction averages may sum exactly national averages covid19 associated deaths factors disease chronic conditions may also contributed caused deaths associated covid19 national notifiable diseases surveillance system nndss death data may delay two weeks confirming death notifications publicly reported


100%|███████████████████████████████████████| 1755/1755 [00:44<00:00, 39.63it/s]


In [ ]:
embedding_dict = pickle.load(open(f'./embedding_dicts_{dataset}_fold_0.pkl','rb'))


In [ ]:
claim_feats, claim_labels, claim_texts = [], [], []
claim_ori_ids = []
claim_ids = {}
claim_labels = []
src_support_claims = defaultdict(set)
src_not_support_claims = defaultdict(set)

for _, record in tqdm(df.iterrows()):
    if record.id_left not in claim_ids:
        claim_texts.append(record.claim_text)
        claim_feats.append(embedding_dict[record.id_left])
        claim_labels.append(int(record.cred_label))
        claim_ids[record.id_left] = len(claim_ids)
        claim_ori_ids.append(record.id_left)
    claim_id = claim_ids[record.id_left]

    if record.score < 0.5:
        src_not_support_claims[record.evidence_source].add(claim_id)
    else:
        src_support_claims[record.evidence_source].add(claim_id)
claim_features = np.array(claim_feats)
claim_labels = np.array(claim_labels)
claim_ori_ids = np.array(claim_ori_ids)

1755it [00:00, 14299.71it/s]


In [ ]:
for src in list(src_support_claims):
    src_support_claims[src] = list(src_support_claims[src])
for src in list(src_not_support_claims):
    src_not_support_claims[src] = list(src_support_claims[src])

In [ ]:
edges = defaultdict(set)
for src in src_support_claims:
    claims =  src_support_claims[src]
    for i in range(len(claims)):
        for j in range(i + 1, len(claims)):
            edges['1-1'].add((claims[i], claims[j]))
            edges['1-1'].add((claims[j], claims[i]))
for src in src_not_support_claims:
    claims = src_not_support_claims[src]
    for i in range(len(claims)):
        for j in range(i + 1, len(claims)):
            edges['0-0'].add((claims[i], claims[j]))
            edges['0-0'].add((claims[j], claims[i]))
for src in set(src_not_support_claims).intersection(src_support_claims):
    claims_1 = src_support_claims[src]
    claims_0 = src_not_support_claims[src]
    for claim_1 in claims_1:
        for claim_0 in claims_0:
            edges['1-0'].add((claim_1, claim_0))
            edges['0-1'].add((claim_0, claim_1))

In [ ]:
for claims_ in src_support_claims.values():
    for i in range(len(claims_)):
        for j in range(i + 1, len(claims_)):
            edges['mutual_evd'].add((claims_[i], claims_[j]))
            edges['mutual_evd'].add((claims_[j], claims_[i]))

In [ ]:
term_to_claim_ids = defaultdict(set)
for claim_id, claim_text in tqdm(enumerate(claim_texts)):
    text = TextBlob(claim_text)
    for term in text.noun_phrases:
        term_to_claim_ids[term].add(claim_id)


351it [00:02, 153.90it/s]


In [ ]:
for claim_ids_ in term_to_claim_ids.values():
    claim_ids_ = list(claim_ids_)
    for i in range(len(claim_ids_)):
        for j in range(i + 1, len(claim_ids_)):
            edges['np'].add((claim_ids_[i], claim_ids_[j]))
            edges['np'].add((claim_ids_[j], claim_ids_[i]))

In [ ]:
for etype in list(edges):
    edges[etype] = np.array([list(e) for e in edges[etype]])

In [ ]:
for i in tqdm(range(5)):
    df_train = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/5fold/train_{i}.tsv', delimiter='\t')
    df_test = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/5fold/test_{i}.tsv', delimiter='\t')
    df_dev = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/dev.tsv', delimiter='\t')
    train_ids, val_ids, test_ids = set(df_train.id_left), set(df_dev.id_left), set(df_test.id_left)
    train_mask, val_mask, test_mask = [], [], []
    for id_left in claim_ori_ids:
        is_train, is_val, is_test = 0, 0, 0
        if id_left in train_ids:
            is_train = 1
        elif id_left in test_ids:
            is_test = 1
        else:
            is_val = 1
        train_mask.append(is_train)
        val_mask.append(is_val)
        test_mask.append(is_test)
        
    graphs = []
    for ignored_rels in [['mutual_evd'], 
                        ['mutual_evd', 'np'],
                        ['1-1', '0-0', '1-0', '0-1'],
                        ['1-1', '0-0', '1-0', '0-1', 'np']]:
        g = dgl.heterograph({
            ('claim', etype, 'claim') : (merged_edges_[:, 0], merged_edges_[:, 1])
            for etype, merged_edges_ in edges.items() if etype not in ignored_rels
        }, num_nodes_dict = {'claim' : len(claim_labels)}).to('cuda')
        g.ndata['train_mask'] = torch.BoolTensor(train_mask).cuda()
        g.ndata['val_mask'] = torch.BoolTensor(val_mask).cuda()
        g.ndata['test_mask'] = torch.BoolTensor(test_mask).cuda()
        g.ndata['label'] = torch.LongTensor(claim_labels).cuda()
        g.ndata['feature'] = torch.FloatTensor(claim_features).cuda()
        graphs.append(g)
    save_graphs(f"../GAGA/pytorch_gaga/preprocessing/{dataset}_{i}.bin", graphs)

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 12.45it/s]
